In [ ]:
class Graph:
    def __init__(self, vertices):
        self.vertices = vertices
        self.edges = []
    
    def add_edge(self, u, v, weight):
        self.edges.append((u, v, weight))

    def bellman_ford(self, source):
        distance = [float('inf')] * self.vertices
        distance[source] = 0

        # relaxation
        for _ in range(self.vertices - 1):
            for u, v, weight in self.edges:
                if distance[u] != float('inf') and distance[u] + weight < distance[v]:
                    distance[v] = distance[u] + weight
        
        # detection
        for u, v, weight in self.edges:
            if distance[u] != float('inf') and distance[u] + weight < distance[v]:
                return "Graph contains a negative-weight cycle"

        return distance


        

In [15]:
distance = [float('inf')] * 5
distance

[inf, inf, inf, inf, inf]

In [23]:
import heapq
import math

def bellman_ford(graph, source, n): # 
    dist = [math.inf] * n
    dist[source] = 0
    
    for _ in range(n - 1): # relaxed these times casue if there at lease only one node connected to all verticess will be relaxxed the same times
        for u in range(n):
            for v,w in graph[u]: # if fully connected graph this alg will be o(v^3)
                if dist[u] + w < dist[v]:
                    dist[v] = dist[u] + w
    for u in range(n):
        for v, w in graph[u]:
            if dist[u] + w < dist[v]:
                raise ValueError("Graph contains a negative weight cycle")
    return dist

def dijkstra(graph, source, n):
    dist = [math.inf] * n
    dist[source] = 0
    pq = [(0, source)]

    while pq:# will use eager dijkstra
        d, u = heapq.heappop(pq)
        if d > dist[u]:
            continue
        for v, w in graph[u]:
            if dist[u] + w  < dist[v]:
                dist[v] = dist[u] + w
                heapq.heappush(pq, (dist[v], v))
    return dist


def johnson(graph, n):
    modified_graph = graph + [[]]  # Add a new vertex with edges to all others
    for i in range(n):
        modified_graph[-1].append((i, 0))
    h = bellman_ford(modified_graph, n, n+1) # n is start from jonson 0 vertex
    reweighted_graph = [[] for _ in range(n)]
    for u in range(n):
        for v, w in graph[u]:
            reweighted_graph[u].append((v, w + h[u] - h[v]))
    all_pairs_shortest_paths = []
    for i in range(n):
        dist = dijkstra(reweighted_graph, i, n)
        for j in range(n):
            if dist[j] < math.inf: # path exsit
                dist[j] += h[j] - h[i]
        all_pairs_shortest_paths.append(dist)

    return all_pairs_shortest_paths  # Return the matrix of all-pairs shortest paths

n = 5
graph = [
    [(1, 3), (2, 8)],    # Vertex 0: edges to 1 with weight 3, 2 with weight 8
    [(2, 2), (3, 1)],    # Vertex 1: edges to 2 with weight 2, 3 with weight 1
    [(4, 4)],            # Vertex 2: edge to 4 with weight 4
    [(2, -5), (4, 2)],   # Vertex 3: edge to 2 with weight -5, 4 with weight 2
    []                   # Vertex 4: no outgoing edges
]


try:
    result = johnson(graph, n)  # Call the Johnson's algorithm to compute all-pairs shortest paths
    for i in range(n):
        print(f"Shortest paths from node {i}: {result[i]}")  # Print shortest paths from each node
except ValueError as e:
    print(e)  # Print error message if a negative weight cycle is detected
    

Shortest paths from node 0: [0, 3, -1, 4, 3]
Shortest paths from node 1: [inf, 0, -4, 1, 0]
Shortest paths from node 2: [inf, inf, 0, inf, 4]
Shortest paths from node 3: [inf, inf, -5, 0, -1]
Shortest paths from node 4: [inf, inf, inf, inf, 0]


In [19]:
graph = [
    [(1, 3), (2, 8)],    # Vertex 0: edges to 1 with weight 3, 2 with weight 8
    [(2, 2), (3, 1)],    # Vertex 1: edges to 2 with weight 2, 3 with weight 1
    [(4, 4)],            # Vertex 2: edge to 4 with weight 4
    [(2, -5), (4, 2)],   # Vertex 3: edge to 2 with weight -5, 4 with weight 2
    []                   # Vertex 4: no outgoing edges
]

a= [[(i, 0) for i in range(5)]]
a, graph + a

([[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0)]],
 [[(1, 3), (2, 8)],
  [(2, 2), (3, 1)],
  [(4, 4)],
  [(2, -5), (4, 2)],
  [],
  [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0)]])

In [ ]:
def floyd_warshall(graph):
    v = len(graph)
    dist = [[float('inf')] * v for _ in range(v)]

    for i in range(v):
        for j in range(v):
            if i == j:
                dist[i][j] = 0
            elif graph[i][j] != 0:
                dist[i][j] = graph[i][j]
    
    for k in range(v):
        for i in range(v):
            for j in range(v):
                dist[i][j] = min(dist[i][j] , dist[i][k] + dist[k][j])
    return dist


In [24]:
import sys

def tsp(n, dist):
    dp = [[sys.maxsize] * n for _ in range(1 << n)]
    """
sys.maxsize is part of the sys module and is typically used to represent the 
maximum size of a container, like a list or an array. It’s also commonly used 
as an indicator of infinity when implementing algorithms that need a large 
placeholder value (like in Dynamic Programming, where it might represent an 
initially "infinite" cost or distance).    
    """
    # Starting at city 0
    dp[1][0] = 0
    
    for mask in range(1 << n):  # 1 << n is 2^n, representing all subsets of cities
        for u in range(n):
            if (mask & (1 << u)) == 0:
                continue
            for v in range(n):
                if mask & (1 << v):
                    continue
                next_mask = mask | (1 << v)
                # Update the DP table with the minimum cost for visiting cities in next_mask, 
                  # ending at city 'v'
                dp[next_mask][v] = min(dp[next_mask][v], dp[mask][u] + dist[u][v])
    
    final_cost = sys.maxsize
    for i in range(1, n): 

        final_cost = min(final_cost, dp[(1 << n) - 1][i] + dist[i][0])

    return final_cost, dp


In [ ]:
from collections import defaultdict

class EularianGraph:
    def __init__(self, graph):
        self.graph = graph
        self.in_degree = defaultdict{int}
        self.out_degree = defaultdict{int}
        self.edge_count = 0
        self.initialize_degrees()

    def _initilize_degrees(self):
        for node in self.graph:
            for neighbor in self.graph[node]:
                self.in_degree[neighbor] += 1
                self.out_degree[neighbor] += 1
                self.edge_count += 1
    
    def find_start_node(self):
        start_nodes, end_nodes, start_node = 0, 0, None

        for node in self.graph:
            out_diff_in  = self.out_degree[node] - self.in_degree[node]
            in_diff_out  = self.in_degree[node]  - self.out_degree[node]

            if abs(out_diff_in) > 1 or abs(in_diff_out) > 1:
                return None 
            
            if out_diff_in == 1:
                start_nodes += 1
                start_node = node
            elif in_diff_out == 1:
                end_nodes += 1
            
        # it could be eular circuit so, 
        if start_nodes == 0 and end_nodes == 0:
            for node in self.graph:
                if self.out_degree[node] > 0:
                    return node
        elif start_nodes == 1 and end_nodes == 1:
            return start_node
        return None
    
    def eulerian_parth_or_circuit(self):
        start_node = self.find_start_node()
        if start_node is None:
            return None
        
        path = []
        self._dfs(start_node, path)

        if len(path) == self.edge_count + 1:
            return path
        else:
            return None
        
    def _dfs(self, node, path):
        while self.out_degree[node] > 0:
            next_node = self.graph[node].pop()
            self.out_degree[node] -= 1
            self._dfs(next_node, path)
        path.append(node)


In [ ]:
import heapq

def laze_prim(graph, start):
    mst = []
    visited = set()
    min_heap = [(0, start)]
    while min_heap:
        weight, vertex = heapq.heappop(min_heap)
        if vertex not  in visited:
            visited.add(vertex)
            if weight > 0:
                mst.append((weight, vertex))
            for neighbor, edge_weight in graph[vertex]:
                if neighbor not in visited:
                    heapq.heappush(min_heap, (edge_weight, neighbor))
    return mst

In [ ]:
import heapq

def prim(graph, start):
    mst = [] 
    visited = set()  
    min_heap = [(0, start)]  

    while min_heap:
        weight, vertex = heapq.heappop(min_heap)
        
        if vertex not in visited:
            visited.add(vertex)
            if weight > 0: 
                mst.append((weight, vertex))

            for neighbor, edge_weight in graph[vertex]:
                if neighbor not in visited:
                    heapq.heappush(min_heap, (edge_weight, neighbor))
            visited.add(vertex)

    return mst


In [ ]:
def eager_prim(graph, start):
    mst = []
    visited = set()
    min_heap = []

    def add_edges(node):
        visited.add(node)
        for neighbor, weight in graph[node]:
            if neighbor not in visited:
                heapq.heappush(min_heap,(weight, node, neighbor))
    add_edges(start)

    while min_heap:
        weight, u, v = heapq.heappop(min_heap)

        if v not in visited:
            mst.append((u, v, weight))
            add_edges(v)
    return mst
    